In [206]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import (
    MinMaxScaler,
    LabelEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    StandardScaler
)
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    confusion_matrix,
    classification_report,
)
from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
    LassoCV,
    RidgeCV,
    LogisticRegression,
)
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier

In [157]:
df = pd.read_csv("data/phones.csv")

In [158]:
TARGET = "Launched Price (USA)"
df.head()

,Company Name,Model Name,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
0,Apple,iPhone 16 128GB,NaN,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,USD 799,2024
1,Apple,iPhone 16 256GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,USD 849,2024
2,Apple,iPhone 16 512GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,USD 899,2024
3,NaN,iPhone 16 Plus 128GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,USD 899,2024
4,NaN,iPhone 16 Plus 256GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,USD 949,2024


In [159]:
df.isna().sum()

Company Name             76
Model Name                0
Mobile Weight           197
RAM                       0
Front Camera              0
Back Camera               0
Processor                 0
Battery Capacity          0
Screen Size               0
Launched Price (USA)      9
Launched Year             0
dtype: int64

In [160]:
df['Company Name'].unique()

array(['Apple', nan, 'Samsung', 'OnePlus', 'Vivo', 'iQOO', 'Oppo',
       'Realme', 'Xiaomi', 'Lenovo', 'Motorola', 'Huawei', 'Nokia',
       'Sony', 'Google', 'Tecno', 'Infinix', 'Honor', 'POCO'],
      dtype=object)

In [ ]:
df[df['Company Name'].isna()].head()

,Company Name,Model Name,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
3,NaN,iPhone 16 Plus 128GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,USD 899,2024
4,NaN,iPhone 16 Plus 256GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,USD 949,2024
57,NaN,iPhone 12 Pro Max 128GB,228g,6GB,12MP / 4K,12MP + 12MP + 12MP,A14 Bionic,"3,687mAh",6.7 inches,"USD 1,099",2020
76,NaN,iPhone XS Max 512GB,208g,4GB,7MP,12MP + 12MP,A12 Bionic,"3,174mAh",6.5 inches,"USD 1,399",2018
86,NaN,iPad Pro 11-inch 128GB,468g,4GB,7MP,12MP + 10MP,A12Z Bionic,"7,812mAh",11 inches,USD 799,2020


In [ ]:
count = 0
for row in df["Model Name"]:
    if "GB" not in row:
        count += 1
        continue
    a = re.findall(r"\d+GB", row)[0].split("GB")[0]
count

157

In [163]:
# models = df[df['Company Name'].isna()]["Model Name"].values.tolist()
# models
company_dict = {}
for company, model in zip(df["Company Name"], df["Model Name"]):
    if pd.isna(company) or pd.isna(model): continue
    company_dict[model.split(" ")[0]] = company
company_dict

{'iPhone': 'Apple',
 'iPad': 'Apple',
 'Galaxy': 'Samsung',
 'OnePlus': 'OnePlus',
 'X200': 'Vivo',
 'V40e': 'Vivo',
 'Y200': 'Vivo',
 'T3': 'Vivo',
 'Y100': 'Vivo',
 'S18': 'Vivo',
 'V30': 'Vivo',
 'iQOO': 'Vivo',
 'Z3': 'Vivo',
 'V9': 'Vivo',
 'X9': 'Vivo',
 'X5Max': 'Vivo',
 'V5': 'Vivo',
 'Y95': 'Vivo',
 'V3': 'Vivo',
 'X3S': 'Vivo',
 'Y66': 'Vivo',
 'V19': 'Vivo',
 'V17': 'Vivo',
 'Y12s': 'Vivo',
 'S1': 'Vivo',
 'Y11': 'Vivo',
 'V15': 'Vivo',
 'X27': 'Vivo',
 'Y30': 'Vivo',
 'Z1': 'Vivo',
 'X21': 'Vivo',
 'V23e': 'Vivo',
 'Y33s': 'Vivo',
 'X60': 'Honor',
 'V20': 'Vivo',
 'Y75': 'Vivo',
 'Y53s': 'Vivo',
 'X70': 'Vivo',
 'T1': 'Vivo',
 'X30': 'Honor',
 'V27': 'Vivo',
 'V25': 'Vivo',
 'X90': 'Vivo',
 'T2': 'Vivo',
 'V23': 'Vivo',
 'X80': 'Vivo',
 'Y21': 'Vivo',
 'Pad': 'Honor',
 'Find': 'Oppo',
 'Reno13': 'Oppo',
 'Reno12': 'Oppo',
 'Reno11': 'Oppo',
 'A5': 'Oppo',
 'A80': 'Oppo',
 'A3': 'Oppo',
 'A3x': 'Oppo',
 'A60': 'Oppo',
 'K12': 'Oppo',
 'F27': 'Oppo',
 'F25': 'Oppo',
 'Reno10'

In [164]:
models = df["Model Name"]
company_names = df["Company Name"]
for i, model in enumerate(models):
    if pd.isna(company_names[i]):
        first = model.split(" ")[0]
        if first in company_dict:
            company_names[i] = company_dict[first]

df["Company Name"] = company_names


/var/folders/g8/z4hc_3fn2m31gh607khncy_h0000gn/T/ipykernel_38419/1953359697.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_names[i] = company_dict[first]
/var/folders/g8/z4hc_3fn2m31gh607khncy_h0000gn/T/ipykernel_38419/1953359697.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_names[i] = company_dict[first]
/var/folders/g8/z4hc_3fn2m31gh607khncy_h0000gn/T/ipykernel_38419/1953359697.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [165]:
df.dropna(subset=["Company Name"], inplace=True)

In [166]:
df.isna().sum()

Company Name              0
Model Name                0
Mobile Weight           195
RAM                       0
Front Camera              0
Back Camera               0
Processor                 0
Battery Capacity          0
Screen Size               0
Launched Price (USA)      9
Launched Year             0
dtype: int64

In [167]:
df.drop(columns=["Mobile Weight"], inplace=True)

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 927 entries, 0 to 929
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company Name          927 non-null    object
 1   Model Name            927 non-null    object
 2   RAM                   927 non-null    object
 3   Front Camera          927 non-null    object
 4   Back Camera           927 non-null    object
 5   Processor             927 non-null    object
 6   Battery Capacity      927 non-null    object
 7   Screen Size           927 non-null    object
 8   Launched Price (USA)  918 non-null    object
 9   Launched Year         927 non-null    int64 
dtypes: int64(1), object(9)
memory usage: 79.7+ KB


In [169]:
df["Front Camera"].unique()

array(['12MP', '12MP / 4K', '7MP', '10MP', '32MP', '13MP', '5MP', '16MP',
       '8MP', '12MP + 12MP', '2MP', '44MP', '24MP', '20MP+8MP', '20MP',
       '50MP', '25MP', '60MP', '10.7MP', 'Dual 32MP', 'Dual 60MP',
       '60MP (ultrawide) + 8MP (telephoto)', '60MP + 8MP', '11.1MP',
       '10.8MP', '10.5MP', '48MP', '42MP', '10MP, 4MP (UDC)'],
      dtype=object)

In [170]:
df["Back Camera"].unique()

array(['48MP', '50MP + 12MP', '48MP + 12MP', '12MP + 12MP', '12MP',
       '12MP + 12MP + 12MP', '8MP', '12MP + 10MP', '200MP + 12MP',
       '108MP + 12MP', '48MP + 8MP', '50MP + 5MP', '50MP + 2MP',
       '108MP + 8MP', '50MP + 8MP', '13MP + 2MP', '12MP + 16MP', '50MP',
       '16MP', '16MP + 5MP', '13MP', '13MP + 5MP', '16MP + 8MP',
       '13MP + 8MP', '13MP + 6MP', '5MP', '50MP + 48MP', '108MP',
       '64MP + 2MP', '48MP + 48MP', '48MP + 50MP', '48MP + 16MP',
       '48MP + 5MP', '64MP + 8MP', '20MP + 16MP', '16MP + 20MP',
       '50MP + 16MP', '200MP', '64MP', '13MP+2MP', '48MP + 64MP + 48MP',
       '50MP + 32MP + 48MP', '50MP + 50MP + 50MP', '50MP + 50MP',
       '50MP + 8MP + 2MP', '50MP + 50MP + 8MP', '50MP + 32MP + 8MP',
       '64MP + 8MP + 2MP', '8MP + 2MP', '50MP + 50MP + 64MP',
       '50MP + 50MP + 13MP', '50MP + 48MP + 32MP', '64MP + 32MP + 8MP',
       '50MP + 64MP + 8MP', '64MP + 8MP + 2MP + 2MP',
       '50MP + 13MP + 16MP + 2MP', '50MP + 16MP + 13MP + 2MP',
      

In [171]:
front_camera_values = []
back_camera_values = []

for row in df["Front Camera"]:
    nums = [float(val) for val in re.findall(r"\d+\.?\d*", row)]
    front_camera_values.append(max(nums))

for row in df["Back Camera"]:
    nums = [float(val) for val in re.findall(r"\d+\.?\d*", row)]
    back_camera_values.append(max(nums))




df["Front Camera"] = front_camera_values
df["Back Camera"] = back_camera_values


In [172]:
df.head()

,Company Name,Model Name,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
0,Apple,iPhone 16 128GB,6GB,12.0,48.0,A17 Bionic,"3,600mAh",6.1 inches,USD 799,2024
1,Apple,iPhone 16 256GB,6GB,12.0,48.0,A17 Bionic,"3,600mAh",6.1 inches,USD 849,2024
2,Apple,iPhone 16 512GB,6GB,12.0,48.0,A17 Bionic,"3,600mAh",6.1 inches,USD 899,2024
3,Apple,iPhone 16 Plus 128GB,6GB,12.0,48.0,A17 Bionic,"4,200mAh",6.7 inches,USD 899,2024
4,Apple,iPhone 16 Plus 256GB,6GB,12.0,48.0,A17 Bionic,"4,200mAh",6.7 inches,USD 949,2024


In [173]:
def process_ram(val):
    return float(val.split("GB")[0])

df["RAM"] = df["RAM"].apply(process_ram)
df.head()

,Company Name,Model Name,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
0,Apple,iPhone 16 128GB,6.0,12.0,48.0,A17 Bionic,"3,600mAh",6.1 inches,USD 799,2024
1,Apple,iPhone 16 256GB,6.0,12.0,48.0,A17 Bionic,"3,600mAh",6.1 inches,USD 849,2024
2,Apple,iPhone 16 512GB,6.0,12.0,48.0,A17 Bionic,"3,600mAh",6.1 inches,USD 899,2024
3,Apple,iPhone 16 Plus 128GB,6.0,12.0,48.0,A17 Bionic,"4,200mAh",6.7 inches,USD 899,2024
4,Apple,iPhone 16 Plus 256GB,6.0,12.0,48.0,A17 Bionic,"4,200mAh",6.7 inches,USD 949,2024


In [174]:
df.drop(columns=["Processor"], inplace=True)

In [175]:
def process_screen_size(val):
    return float(val.split(" inches")[0])
df["Screen Size"] = df["Screen Size"].apply(process_screen_size) 

In [176]:
def process_battery(val):
    return float(val.replace(",","").split("mAh")[0])
df["Battery Capacity"] = df["Battery Capacity"].apply(process_battery) 

In [177]:
df.dropna(subset=["Launched Price (USA)"], inplace=True)

In [178]:
def process_price(val):
    return float(val.replace(",","").split("USD ")[1])
df["Launched Price (USA)"] = df["Launched Price (USA)"].apply(process_price) 

In [188]:
len(df["Model Name"].unique())/len(df)

0.9760348583877996

In [189]:
df.drop(columns=["Model Name"], inplace=True)

In [ ]:
df.head()

,Company Name,RAM,Front Camera,Back Camera,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
0,Apple,6.0,12.0,48.0,3600.0,6.1,799.0,2024
1,Apple,6.0,12.0,48.0,3600.0,6.1,849.0,2024
2,Apple,6.0,12.0,48.0,3600.0,6.1,899.0,2024
3,Apple,6.0,12.0,48.0,4200.0,6.7,899.0,2024
4,Apple,6.0,12.0,48.0,4200.0,6.7,949.0,2024


In [191]:
len(df["Company Name"].unique())

18

In [192]:
df["Company Name"].unique()

array(['Apple', 'Samsung', 'OnePlus', 'Vivo', 'Honor', 'iQOO', 'Oppo',
       'Lenovo', 'Realme', 'Xiaomi', 'Motorola', 'Huawei', 'Nokia',
       'Sony', 'Google', 'Tecno', 'Infinix', 'POCO'], dtype=object)

In [195]:
df.head()

,Company Name,RAM,Front Camera,Back Camera,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
0,Apple,6.0,12.0,48.0,3600.0,6.1,799.0,2024
1,Apple,6.0,12.0,48.0,3600.0,6.1,849.0,2024
2,Apple,6.0,12.0,48.0,3600.0,6.1,899.0,2024
3,Apple,6.0,12.0,48.0,4200.0,6.7,899.0,2024
4,Apple,6.0,12.0,48.0,4200.0,6.7,949.0,2024


In [ ]:
one_hot_encoded_values = pd.get_dummies(df["Company Name"])
df = pd.concat([df, one_hot_encoded_values], axis=1)


In [198]:
df.drop(columns="Company Name", inplace=True)

In [201]:
df.head()

,RAM,Front Camera,Back Camera,Battery Capacity,Screen Size,Launched Price (USA),Launched Year,Apple,Google,Honor,...,OnePlus,Oppo,POCO,Realme,Samsung,Sony,Tecno,Vivo,Xiaomi,iQOO
0,6.0,12.0,48.0,3600.0,6.1,799.0,2024,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,6.0,12.0,48.0,3600.0,6.1,849.0,2024,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,6.0,12.0,48.0,3600.0,6.1,899.0,2024,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6.0,12.0,48.0,4200.0,6.7,899.0,2024,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,6.0,12.0,48.0,4200.0,6.7,949.0,2024,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [203]:
X, y = df.drop(columns=["Launched Price (USA)"]), df["Launched Price (USA)"]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [207]:
regressor = XGBRegressor(
    n_estimators=100
).fit(train_X, train_y)

pred_y = regressor.predict(test_X)

print(f"MAE: {mean_absolute_error(test_y, pred_y)}")
print(f"MAPE: {mean_absolute_percentage_error(test_y, pred_y)}")
print(f"MSE: {mean_squared_error(test_y, pred_y)}")
print(f"RMSE: {root_mean_squared_error(test_y, pred_y)}")
print(f"R2 Score: {r2_score(test_y, pred_y)}")

MAE: 66.73772555641506
MAPE: 0.13571112585523726
MSE: 11069.285476681196
RMSE: 105.21067187638903
R2 Score: 0.9414315163428075


In [208]:
regressor = RandomForestRegressor(
    n_estimators=100,
).fit(train_X, train_y)

pred_y = regressor.predict(test_X)

print(f"MAE: {mean_absolute_error(test_y, pred_y)}")
print(f"MAPE: {mean_absolute_percentage_error(test_y, pred_y)}")
print(f"MSE: {mean_squared_error(test_y, pred_y)}")
print(f"RMSE: {root_mean_squared_error(test_y, pred_y)}")
print(f"R2 Score: {r2_score(test_y, pred_y)}")

MAE: 78.61162524197722
MAPE: 0.16972560618995677
MSE: 15147.316045077565
RMSE: 123.0744329463986
R2 Score: 0.9198543271735691
